In [29]:
import gmsh
gmsh.initialize()
using LinearAlgebra, SparseArrays

In [30]:
# Data
height = 100
base = 100
elemSize = 100

100

In [31]:
gmsh.model.add("rectangle")

p1 = gmsh.model.occ.addPoint(-1, -1, 0)
p2 = gmsh.model.occ.addPoint(1, -1, 0)
p3 = gmsh.model.occ.addPoint(1, 1, 0)
p4 = gmsh.model.occ.addPoint(-1, 1, 0)

l1 = gmsh.model.occ.addLine(p1, p2)
l2 = gmsh.model.occ.addLine(p2, p3)
l3 = gmsh.model.occ.addLine(p3, p4)
l4 = gmsh.model.occ.addLine(p4, p1)

cl1 = gmsh.model.occ.addCurveLoop([l1, l2, l3, l4])

sf1 = gmsh.model.occ.addPlaneSurface([cl1])

gmsh.model.occ.synchronize()

In [32]:
supp1 = gmsh.model.addPhysicalGroup(1, [l1])
gmsh.model.setPhysicalName(1, supp1, "FixBottomY")
supp2 = gmsh.model.addPhysicalGroup(1, [l4])
gmsh.model.setPhysicalName(1, supp2, "FixLeftX")
trac1 = gmsh.model.addPhysicalGroup(1, [l3])
gmsh.model.setPhysicalName(1, trac1, "ForceTopY")

In [33]:
all = gmsh.model.getEntities(0);
gmsh.model.mesh.setSize(all, elemSize) # element size
gmsh.model.mesh.setAlgorithm(2, sf1, 8)
gmsh.model.mesh.generate(1)
gmsh.model.mesh.generate(2)
gmsh.model.mesh.recombine()
# másodfokú elemekhez:
#gmsh.option.setNumber("Mesh.SecondOrderIncomplete", 1)
#gmsh.model.mesh.setOrder(2)

gmsh.option.setNumber("Geometry.CurveLabels", 1)
gmsh.option.setNumber("Mesh.Points", 1)
gmsh.option.setNumber("Mesh.Lines", 1)
gmsh.option.setNumber("Mesh.NodeLabels", 1)
gmsh.option.setNumber("Mesh.LineLabels", 1)
gmsh.option.setNumber("Mesh.SurfaceLabels", 1)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.00234304s, CPU 0.002123s)
Info    : 4 nodes 8 elements
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay for Quads)
Info    : Done meshing 2D (Wall 0.00149058s, CPU 0.001484s)
Info    : 4 nodes 10 elements
Info    : Recombining 2D mesh...
Info    : Blossom: 1 internal 2 closed
Info    : Blossom recombination completed (Wall 0.000122031s, CPU 0.000119s): 1 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 1, min Q = 1
Info    : Done recombining 2D mesh (Wall 0.000180732s, CPU 0.000186s)


In [34]:
# SA
E = 2e5
ν = 0.3
D = E / ((1 + ν) * (1 - 2ν)) * [1-ν ν 0; ν 1-ν 0; 0 0 (1-2ν)/2]

3×3 Matrix{Float64}:
 2.69231e5  1.15385e5      0.0
 1.15385e5  2.69231e5      0.0
 0.0        0.0        76923.1

In [35]:
# ÁSF
E = 2e5
ν = 0.3
D = E / (1 - ν^2) * [1 ν 0; ν 1 0; 0 0 (1-ν)/2]

3×3 Matrix{Float64}:
     2.1978e5  65934.1           0.0
 65934.1           2.1978e5      0.0
     0.0           0.0       76923.1

In [36]:
# Merevségi mátrix felépítése
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(2, -1)
nn = []
I = []
J = []
V = []
V = convert(Vector{Float64}, V)
#sizehint!(I, 8 * 8 * noe)
#sizehint!(J, 8 * 8 * noe)
#sizehint!(V, 8 * 8 * noe)
for i in 1:length(elemTypes)
    et = elemTypes[i]
    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
    intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(et, "Gauss3")
    numIntPoints = length(intWeights)
    comp, fun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
    h = reshape(fun, :, numIntPoints)
    comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
    #display(ori)
    ∇h = reshape(dfun, :, numIntPoints)
    nnet = zeros(Int, length(elemTags[i]), numNodes)
    for j in 1:length(elemTags[i])
        elem = elemTags[i][j]
        for k in 1:numNodes
            nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
        end
        jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
        Jac = reshape(jac, 3, :)
        #display(reshape(coord, 3, :))
        invJac = similar(Jac)
        for k in 1:numIntPoints
            invJac[1:3, 3*k-2:3*k] = inv(Jac[1:3, 3*k-2:3*k])
        end
        ∂h = zeros(3, numNodes * numIntPoints)
        for k in 1:numIntPoints, l in 1:numNodes
            ∂h[1:3, (k-1)*numIntPoints+l] = invJac[1:3, k*3-2:k*3] * ∇h[l*3-2:l*3, k]
        end
        #display(∂h)
        B = zeros(3 * numIntPoints, 2 * numNodes)
        for k in 1:numIntPoints
            for l in 1:numNodes
                B[k*3-0, l*2-0] = B[k*3-2, l*2-1] = ∂h[1, (k-1)*numIntPoints+l]
                B[k*3-0, l*2-1] = B[k*3-1, l*2-0] = ∂h[2, (k-1)*numIntPoints+l]
            end
        end
        K1 = zeros(2 * numNodes, 2 * numNodes)
        for k in 1:numIntPoints
            B1 = B[k*3-2:k*3, 1:2*numNodes]
            #display(B1)
            K1 += B1' * D * B1 * jacDet[k] * intWeights[k]
        end
        #display(K1)
        Iidx = zeros(Int, numNodes * 2, numNodes * 2)
        Jidx = zeros(Int, numNodes * 2, numNodes * 2)
        for k in 1:numNodes*2, l in 1:numNodes*2
            Iidx[k, l] = k
            Jidx[k, l] = l
        end
        nn2 = zeros(Int, 2 * numNodes)
        nn2[1:2:2*numNodes] = 2 * nnet[j, 1:numNodes] .- 1
        nn2[2:2:2*numNodes] = 2 * nnet[j, 1:numNodes]
        #display(nn2)
        append!(I, nn2[Iidx[:]])
        append!(J, nn2[Jidx[:]])
        append!(V, K1[:])
    end
    push!(nn, nnet)
end
K = sparse(I, J, V)
#display(I')
#display(J')
#display(V')
K0 = copy(K)


8×8 SparseMatrixCSC{Float64, Int64} with 64 stored entries:
  98901.1    35714.3   -60439.6   …  -35714.3    10989.0     2747.25
  35714.3    98901.1     2747.25     -49450.5    -2747.25  -60439.6
 -60439.6     2747.25   98901.1       -2747.25  -49450.5    35714.3
  -2747.25   10989.0   -35714.3      -60439.6    35714.3   -49450.5
 -49450.5   -35714.3    10989.0       35714.3   -60439.6    -2747.25
 -35714.3   -49450.5    -2747.25  …   98901.1     2747.25   10989.0
  10989.0    -2747.25  -49450.5        2747.25   98901.1   -35714.3
   2747.25  -60439.6    35714.3       10989.0   -35714.3    98901.1

In [37]:
intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(3, "Gauss3")
numIntPoints = length(intWeights)
intPoints = [-1.0, -0.9, 0, -1.0, 1, 0, 1.0, -1, 0, 1.0, 1.0, 0]
comp, fun, ori = gmsh.model.mesh.getBasisFunctions(3, intPoints, "Lagrange", [0])
display(reshape(fun, :, numIntPoints))
comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(3, intPoints, "GradLagrange", [0])
display(reshape(dfun, :, numIntPoints))
#display(dfun)
jacobians, determinants, coord = gmsh.model.mesh.getJacobian(9, intPoints)
display(reshape(coord, 3, :))
display(reshape(jacobians, :, numIntPoints * 3))

4×4 Matrix{Float64}:
 0.95  0.0  0.0  0.0
 0.0   0.0  1.0  0.0
 0.0   0.0  0.0  1.0
 0.05  1.0  0.0  0.0

12×4 Matrix{Float64}:
 -0.475   0.0  -0.5   0.0
 -0.5    -0.5   0.0   0.0
  0.0     0.0   0.0   0.0
  0.475   0.0   0.5   0.0
  0.0     0.0  -0.5  -0.5
  0.0     0.0   0.0   0.0
  0.025   0.5   0.0   0.5
  0.0     0.0   0.5   0.5
  0.0     0.0   0.0   0.0
 -0.025  -0.5   0.0  -0.5
  0.5     0.5   0.0   0.0
  0.0     0.0   0.0   0.0

3×4 Matrix{Float64}:
 -1.0  -1.0   1.0  1.0
 -0.9   1.0  -1.0  1.0
  0.0   0.0   0.0  0.0

3×12 Matrix{Float64}:
 1.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0

In [38]:
# Teher vektor felépítése
p = [0, 1]
dof, dof = size(K)
f = zeros(dof)
tags = gmsh.model.getEntitiesForPhysicalGroup(1, trac1)
for i ∈ 1:length(tags)
    elementTypes, elementTags, elemNodeTags = gmsh.model.mesh.getElements(1, tags[i])
    if length(elementTypes) != 1
        error("A peremen nem csak egyfajta végeselem van!")
    end
    elements = elementTags[1]
    nnoe = reshape(elemNodeTags[1], 2, :)'
    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(elementTypes[1])
    intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(elementTypes[1], "Gauss3")
    numIntPoints = length(intWeights)
    comp, fun, ori = gmsh.model.mesh.getBasisFunctions(elementTypes[1], intPoints, "Lagrange")
    h = reshape(fun, :, numNodes)
    comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(elementTypes[1], intPoints, "GradLagrange")
    ∇h = reshape(dfun, :, numNodes)
    #nnet = zeros(Int, length(elemTags[i]), numNodes)
    #nnet = zeros(Int, numNodes)
    H = zeros(2 * numIntPoints, 2 * numNodes)
    for j in 1:numIntPoints
        for k in 1:numNodes
            H[j*2-1, k*2-1] = h[j, k]
            H[j*2-0, k*2-0] = h[j, k]
        end
    end
    for l in 1:length(elementTags[1])
        elem = elementTags[1][l]
        jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
        Jac = reshape(jac, 3, :)
        f1 = zeros(2 * numNodes)
        for j in 1:numIntPoints
            H1 = H[j*2-1:j*2, 1:2*numNodes]
            Ja = √((Jac[1, 3*j-2])^2 + (Jac[2, 3*j-2])^2)
            f1 += H1' * p * Ja * intWeights[j]
        end
        nn2 = zeros(Int, 2 * numNodes)
        nn2[1:2:2*numNodes] = 2 * nnoe[l, 1:numNodes] .- 1
        nn2[2:2:2*numNodes] = 2 * nnoe[l, 1:numNodes]
        f[nn2] += f1
    end
end
display(f)

8-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 1.0

In [39]:
nodeTags, coord = gmsh.model.mesh.getNodesForPhysicalGroup(1, supp1)
nodeTags *= 2
display(nodeTags)
display(K)
for i ∈ nodeTags
    K[i, :] .= 0
    K[:, i] .= 0
    K[i, i] = 1
    f[i] = 0
end
dropzeros!(K)
display(K)

2-element Vector{UInt64}:
 0x0000000000000002
 0x0000000000000004

8×8 SparseMatrixCSC{Float64, Int64} with 64 stored entries:
  98901.1    35714.3   -60439.6   …  -35714.3    10989.0     2747.25
  35714.3    98901.1     2747.25     -49450.5    -2747.25  -60439.6
 -60439.6     2747.25   98901.1       -2747.25  -49450.5    35714.3
  -2747.25   10989.0   -35714.3      -60439.6    35714.3   -49450.5
 -49450.5   -35714.3    10989.0       35714.3   -60439.6    -2747.25
 -35714.3   -49450.5    -2747.25  …   98901.1     2747.25   10989.0
  10989.0    -2747.25  -49450.5        2747.25   98901.1   -35714.3
   2747.25  -60439.6    35714.3       10989.0   -35714.3    98901.1

8×8 SparseMatrixCSC{Float64, Int64} with 38 stored entries:
  98901.1    ⋅   -60439.6    ⋅   -49450.5   -35714.3    10989.0     2747.25
       ⋅    1.0        ⋅     ⋅         ⋅          ⋅          ⋅          ⋅ 
 -60439.6    ⋅    98901.1    ⋅    10989.0    -2747.25  -49450.5    35714.3
       ⋅     ⋅         ⋅    1.0        ⋅          ⋅          ⋅          ⋅ 
 -49450.5    ⋅    10989.0    ⋅    98901.1    35714.3   -60439.6    -2747.25
 -35714.3    ⋅    -2747.25   ⋅    35714.3    98901.1     2747.25   10989.0
  10989.0    ⋅   -49450.5    ⋅   -60439.6     2747.25   98901.1   -35714.3
   2747.25   ⋅    35714.3    ⋅    -2747.25   10989.0   -35714.3    98901.1

In [40]:
nodeTags, coord = gmsh.model.mesh.getNodesForPhysicalGroup(1, supp2)
nodeTags *= 2
nodeTags .-= 1
display(K)
for i ∈ nodeTags
    K[i, :] .= 0
    K[:, i] .= 0
    K[i, i] = 1
    f[i] = 0
end
dropzeros!(K)
display(K)

8×8 SparseMatrixCSC{Float64, Int64} with 38 stored entries:
  98901.1    ⋅   -60439.6    ⋅   -49450.5   -35714.3    10989.0     2747.25
       ⋅    1.0        ⋅     ⋅         ⋅          ⋅          ⋅          ⋅ 
 -60439.6    ⋅    98901.1    ⋅    10989.0    -2747.25  -49450.5    35714.3
       ⋅     ⋅         ⋅    1.0        ⋅          ⋅          ⋅          ⋅ 
 -49450.5    ⋅    10989.0    ⋅    98901.1    35714.3   -60439.6    -2747.25
 -35714.3    ⋅    -2747.25   ⋅    35714.3    98901.1     2747.25   10989.0
  10989.0    ⋅   -49450.5    ⋅   -60439.6     2747.25   98901.1   -35714.3
   2747.25   ⋅    35714.3    ⋅    -2747.25   10989.0   -35714.3    98901.1

8×8 SparseMatrixCSC{Float64, Int64} with 20 stored entries:
 1.0   ⋅        ⋅     ⋅        ⋅         ⋅     ⋅        ⋅ 
  ⋅   1.0       ⋅     ⋅        ⋅         ⋅     ⋅        ⋅ 
  ⋅    ⋅   98901.1    ⋅   10989.0   -2747.25   ⋅   35714.3
  ⋅    ⋅        ⋅    1.0       ⋅         ⋅     ⋅        ⋅ 
  ⋅    ⋅   10989.0    ⋅   98901.1   35714.3    ⋅   -2747.25
  ⋅    ⋅   -2747.25   ⋅   35714.3   98901.1    ⋅   10989.0
  ⋅    ⋅        ⋅     ⋅        ⋅         ⋅    1.0       ⋅ 
  ⋅    ⋅   35714.3    ⋅   -2747.25  10989.0    ⋅   98901.1

In [41]:
q = K \ f

8-element Vector{Float64}:
  0.0
  0.0
 -3.0e-6
  0.0
 -2.999999999999999e-6
  9.999999999999999e-6
  0.0
  1.0e-5

In [42]:
p[2] / (E / height)

0.0005

In [43]:
K0 * q

8-element Vector{Float64}:
 -6.591949208711867e-17
 -1.0
  0.0
 -0.9999999999999999
  1.214306433183765e-16
  1.0
 -1.1102230246251565e-16
  0.9999999999999999

In [44]:
k = base * E / height

200000.0

In [45]:
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
non = length(nodeTags)
uvec = gmsh.view.add("uvec")
u = [zeros(3) for i in 1:non]
for i in 1:non, j in 1:2
    u[i][j] = q[(i-1)*2+j]
end

gmsh.view.addModelData(uvec, 0, "rectangle", "NodeData", nodeTags, u)


In [46]:
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
non = length(nodeTags)
ux = gmsh.view.add("ux")
u = [zeros(3) for i in 1:non]
for i in 1:non, j in 1:1
    u[i][j] = q[(i-1)*2+j]
end

gmsh.view.addModelData(ux, 0, "rectangle", "NodeData", nodeTags, u)


In [47]:
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
non = length(nodeTags)
uy = gmsh.view.add("uy")
u = [zeros(3) for i in 1:non]
for i in 1:non, j in 2:2
    u[i][j] = q[(i-1)*2+j]
end

gmsh.view.addModelData(uy, 0, "rectangle", "NodeData", nodeTags, u)


In [48]:
#gmsh.write("rect.geo")
gmsh.write("rect.msh")
gmsh.write("rect.pos")

Info    : Writing 'rect.msh'...
Info    : Done writing 'rect.msh'
Info    : Writing 'rect.pos'...
Info    : Done writing 'rect.pos'


In [49]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20221221
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.6.1
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [50]:
gmsh.finalize()